# 第3章: 役割の割り当て（ロールプロンプティング）

- [レッスン](#レッスン)
- [演習](#演習)
- [サンプルプレイグラウンド](#サンプルプレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

Claudeが言われたこと以外にコンテキストを持たないというテーマを続けると、 **Claudeに特定の役割を与える（必要なコンテキストをすべて含む）** ことが重要な場合があります。これはロールプロンプティングとも呼ばれます。役割のコンテキストが詳細であればあるほど、より良い結果が得られます。

**Claudeに役割を与えることで、Claudeのパフォーマンスを向上させることができます**。これは執筆、コーディング、要約など、さまざまな分野で効果があります。人間が「〇〇のように考える」と言われると助けになることがあるのと同じです。ロールプロンプティングは、Claudeの応答のスタイル、トーン、マナーも変えることができます。

**注意:** ロールプロンプティングは、システムプロンプトまたはユーザーメッセージターンの一部として行うことができます。

### 例

以下の例では、ロールプロンプティングなしでは、スケートボードに関する一文の見解を求められた場合、Claudeは**率直で装飾のない答え**を提供することがわかります。

しかし、Claudeに猫の役割を与えると、Claudeの視点が変わり、**Claudeの応答のトーン、スタイル、内容が新しい役割に適応します**。

**注意:** 使用できるボーナステクニックとして、**Claudeに想定される聴衆に関するコンテキストを提供する**ことがあります。以下では、プロンプトを調整して、Claudeに誰に話しかけるべきかを伝えることもできました。「あなたは猫です」と「あなたはスケートボーダーの群衆に話しかけている猫です」では、かなり異なる応答が得られます。

これはシステムプロンプトでロールプロンプティングを行わない場合のプロンプトです：

In [ ]:
# プロンプト
PROMPT = "一文で、スケートボードについてどう思いますか？"

# Claudeの応答を表示
print(get_completion(PROMPT))

これは同じユーザーの質問ですが、ロールプロンプティングを行っています。

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたは猫です。"

# プロンプト
PROMPT = "一文で、スケートボードについてどう思いますか？"

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))

ロールプロンプティングを使用して、Claudeに特定の書き方のスタイルを模倣させたり、特定の声で話させたり、答えの複雑さを調整したりすることができます。**ロールプロンプティングは、Claudeの数学や論理タスクの性能を向上させることもできます。**

例えば、以下の例では、明確な正解があり、それは「はい」です。しかし、Claudeは間違えて、情報が不足していると考えています（実際には不足していません）：

In [ ]:
# プロンプト
PROMPT = "ジャックはアンを見ています。アンはジョージを見ています。ジャックは結婚しており、ジョージは結婚していません。アンが結婚しているかどうかはわかりません。既婚者が未婚者を見ていますか？"

# Claudeの応答を表示
print(get_completion(PROMPT))

では、**Claudeにロジックボットとして行動するよう指示する**とどうなるでしょうか？Claudeの答えはどのように変わるでしょうか？

この新しい役割割り当てにより、Claudeは正解を出します（ただし、注目すべきは、必ずしもすべての正しい理由によるものではありません）。

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたは複雑な論理問題に答えるように設計されたロジックボットです。"

# プロンプト
PROMPT = "ジャックはアンを見ています。アンはジョージを見ています。ジャックは結婚しており、ジョージは結婚していません。アンが結婚しているかどうかはわかりません。既婚者が未婚者を見ていますか？"

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))

**注意:** このコースを通じて学ぶことは、**同様の結果を得るために使用できるプロンプトエンジニアリング技術が多数ある**ということです。どの技術を使用するかは、あなたの好みによります！**独自のプロンプトエンジニアリングスタイルを見つけるために実験することをお勧めします**。

上記のコンテンツを変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**サンプルプレイグラウンド**](#サンプルプレイグラウンド)をご覧ください。

---

## 演習
- [演習 3.1 - 数学の訂正](#演習-31---数学の訂正)

### 演習 3.1 - 数学の訂正
場合によっては、**Claudeは数学、特に簡単な数学に苦戦することがあります**。以下では、Claudeは数学の問題を正しく解かれたと誤って評価していますが、実際には2番目のステップに明らかな計算ミスがあります。Claudeは実際にステップバイステップで進めるときにミスに気づいていますが、全体的な解答が間違っているという結論には至っていないことに注意してください。

`PROMPT`や`SYSTEM_PROMPT`を修正して、Claudeが解答を正しく解かれたのではなく、`incorrectly`（不正確に）解かれたと評価するようにしてください。


In [ ]:
# システムプロンプト - システムプロンプトを使用したくない場合は、この変数を空の文字列のままにしておくことができます
SYSTEM_PROMPT = ""

# プロンプト
PROMPT = """以下の方程式は正しく解かれていますか？

2x - 3 = 9
2x = 6
x = 3"""

# Claudeの応答を取得
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Claudeの応答と対応する評価を表示
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_3_1_hint)

### おめでとうございます！

ここまですべての演習を解いたなら、次の章に進む準備ができています。プロンプトエンジニアリングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試したり、プロンプトを微調整してClaudeの応答にどのような影響があるかを確認したりできる領域です。

In [ ]:
# プロンプト
PROMPT = "一文で、スケートボードについてどう思いますか？"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたは猫です。"

# プロンプト
PROMPT = "一文で、スケートボードについてどう思いますか？"

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# プロンプト
PROMPT = "ジャックはアンを見ています。アンはジョージを見ています。ジャックは結婚しており、ジョージは結婚していません。アンが結婚しているかどうかはわかりません。既婚者が未婚者を見ていますか？"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたは複雑な論理問題に答えるように設計されたロジックボットです。"

# プロンプト
PROMPT = "ジャックはアンを見ています。アンはジョージを見ています。ジャックは結婚しており、ジョージは結婚していません。アンが結婚しているかどうかはわかりません。既婚者が未婚者を見ていますか？"

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))